In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from qprofiler import DataProfiler
from qprofiler import QTest, QPipeline
from pathlib import Path

In [2]:
# now create an instance of DataProfiler to create a profile for training dataset..
# leave the path parameter will create the profiler directory in current working directory
data_profiler = DataProfiler()

profiler already exists


In [3]:
# now scan the training data to create profile file and save it in .dprofiler.
ref = data_profiler.scan_csv_file("../datasets/loan-perf.csv", unique_identifier="customerid")
# now save it
data_profiler.create_profile(ref,
                             "loan-perf")

profile already exists


In [4]:
# now we have the information that we need to compare new datasets
# to training datasets, and create quality checks...

# let's define an instance of QTest Module
# it takes the path of reference profile.
qtest = QTest(Path(".dprofiler/loan-perf.yml"))

In [5]:
# now create a profile for test dataset.
new = qtest.scan_csv_file("../datasets/loan-perf-test.csv", unique_identifier="customerid")

In [6]:
# create a pipeline of tests
qpipeline = QPipeline(test_obj=qtest, verbose = True)

In [7]:
qpipeline.append(name = "check_columns",
                 check = qtest.check_columns,
                 error_level = "error",
                 test_profile=new)\
         .append(name = "check_unique_categories",
                 check = qtest.check_unique_categories,
                 error_level = "error",
                 test_profile=new)\
         .append(name = "check_numeric_below_thresh",
                 check = qtest.check_numeric_below_thresh,
                 error_level = "warn",
                 test_profile=new,
                 min_thresh = 15, col  = "termdays")

In [8]:
qpipeline.tests

['check_columns', 'check_unique_categories', 'check_numeric_below_thresh']

In [9]:
res = qpipeline.run()

number of columns aren't identical
unique values in test don't match reference
lower bound of termdays is greater than or equal 15.


In [10]:
res

False

In [11]:
qpipeline.remove_step('check_numeric_below_thresh')

check_numeric_below_thresh test removed


In [12]:
qpipeline.tests

['check_columns', 'check_unique_categories']

In [13]:
qpipeline.clear_pipeline()

all quality checks removed


In [14]:
qpipeline.tests

[]

In [13]:
qpipeline.remove_step('check_unique_categories')

ValueError: check_unique_categories test doesn't exist in pipeline